# 🚀 High-Performance ML Pipeline using Transformer Model + XGBoost Multi Label Classifer

This notebook implements a streamlined, high-performance solution for detecting:

- 🚫 **Spam Reviews**: Low-quality, repetitive, or fake reviews
- 🚫 **Advertisements**: Promotional content in reviews
- 🚫 **Rant Without Visit**: Complaints from users who never actually visited

**Approach**:

- **SentenceTransformers** (`all-MiniLM-L6-v2`) for fast 384-dim text embeddings
- **Metadata feature engineering** for comprehensive signals
- **XGBoost** classifiers with hyperparameter tuning
- **Multi-task classification** with optimized F1, precision, and recall

<br>

---


## 📦 1. Setup and Installation

First, let's install required packages and import libraries.


In [7]:
# Install required packages
import subprocess
import sys


def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")
    except:
        print(f"⚠️  {package} installation failed, might already be installed")
        print("bruh")


# Install key packages
packages = [
    "sentence-transformers",
    "xgboost",
    "scikit-learn",
    "pandas",
    "numpy",
    "matplotlib",
    "seaborn",
]

print("📦 Installing required packages...")
for package in packages:
    install_package(package)

print("\n🎉 All packages ready!")

📦 Installing required packages...



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


✅ sentence-transformers installed successfully
✅ xgboost installed successfully



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


✅ scikit-learn installed successfully


...


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


✅ matplotlib installed successfully



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


✅ seaborn installed successfully

🎉 All packages ready!



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
    accuracy_score,
    confusion_matrix,
)
import warnings
import re


# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Set plotting style
plt.style.use("default")
sns.set_palette("husl")

print("🚀 To Build Transformer & XGBoost Classifier Ready!")
print("   ✅ SentenceTransformers for text embeddings")
print("   ✅ Encoding for Metadata Columns")
print("   ✅ XGBoost for high-performance classification")
print("   ✅ Hyperparameter optimization included")
print("   ✅ Model Evaluation too ...")

🚀 To Build Transformer & XGBoost Classifier Ready!
   ✅ SentenceTransformers for text embeddings
   ✅ Encoding for Metadata Columns
   ✅ XGBoost for high-performance classification
   ✅ Hyperparameter optimization included
   ✅ Model Evaluation too ...


## 2. 📊 Data Loading and Preparation

Let's create a realistic dataset that matches your specifications with proper labels.


In [9]:
reviews_df = pd.read_parquet("../data/final_parquet.parquet")
# reviews_df = pd.read_csv("../data/cleaned_google_reviews.csv")
# reviews_df = reviews_df[reviews_df["review_text"].str.len() > 0].reset_index(drop=True)
reviews_df.head(5)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
# Display dataset overview
print(f"\n📈 Dataset Overview:")
print(f"   Shape: {reviews_df.shape}")
print(f"   Columns: {list(reviews_df.columns)}")

print(f"\n📊 Target Distribution:")
target_cols = ["advertisement", "irrelevant", "fake_rant"]
for col in target_cols:
    count = reviews_df[col].sum()
    percentage = reviews_df[col].mean() * 100
    print(f"   {col}: {count:,} ({percentage:.1f}%)")

# # Display sample data
# print(f"\n📝 Sample Data:")
# display_df = reviews_df[["rating", "text", "category"] + target_cols].head()
# for _, row in display_df.iterrows():
#     print(f"\n   Rating: {row['rating']}⭐ | Category: {row['category']}")
#     print(f"   Text: '{row['text'][:80]}...'")
#     violations = [col.replace("is_", "") for col in target_cols if row[col]]
#     print(f"   Labels: {violations if violations else ['Clean']}")

In [ ]:
pd.set_option("display.max_columns", None)
reviews_df.head()

In [ ]:
reviews_df.shape

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def get_combined_embeddings(df, text_columns):
    embeddings_list = []
    for col in text_columns:
        texts = df[col].fillna("").tolist()
        col_embs = model.encode(texts)
        embeddings_list.append(col_embs)
    # Concatenate embeddings horizontally for each row
    return np.hstack(embeddings_list)


text_cols = ["text", "name_x", "name_y"]
combined_embeddings = get_combined_embeddings(reviews_df, text_cols)

In [ ]:
target_cols = ["advertisement", "irrelevant", "fake_rant"]
metadata_cols = reviews_df.drop(columns=text_cols + target_cols).columns.tolist()
print(f"📊 Metadata columns to encode: {metadata_cols}")
print(f"📊 Target columns: {target_cols}")
print(f"🔤 Text columns (already embedded): {text_cols}")

In [ ]:
# ⚡ ULTRA-FAST FEATURE ENGINEERING (OPTIMIZED FOR SPEED)

import time
import re
import numpy as np

start_time = time.time()

print("🚀 ULTRA-FAST FEATURE ENGINEERING PIPELINE...")

# Define column categories based on actual data structure
TARGET_COLUMNS = target_cols
TEXT_COLUMNS = text_cols

print(f"✅ COLUMN IDENTIFICATION:")
print(f"   🎯 Target columns: {TARGET_COLUMNS}")
print(f"   📝 Text columns: {TEXT_COLUMNS}")

# ========================================
# ULTRA-OPTIMIZED FEATURE ENGINEERING
# ========================================


def ultra_fast_feature_engineering(df):
    """Lightning-fast feature engineering optimized for speed"""

    print("   ⚡ Creating features at lightning speed...")

    # Make a copy
    df_fast = df.copy()

    # Map actual column names - fix column name mismatches
    review_text_col = "text" if "text" in df_fast.columns else "review_text"
    resp_text_col = "resp" if "resp" in df_fast.columns else "resp_text"
    time_col = "time" if "time" in df_fast.columns else "review_time"

    # Fill NaN values once at the beginning
    df_fast[review_text_col] = df_fast[review_text_col].fillna("")
    if resp_text_col in df_fast.columns:
        df_fast[resp_text_col] = df_fast[resp_text_col].fillna("")
    df_fast[time_col] = pd.to_datetime(df_fast[time_col], errors="coerce")

    # BATCH 1: Basic numerical features (vectorized operations)
    print("     📊 Basic numerical features...")
    text_series = df_fast[review_text_col]  # Reference once
    df_fast["text_length"] = text_series.str.len()
    df_fast["word_count"] = text_series.str.count(" ") + 1
    df_fast["sentence_count"] = text_series.str.count("[.!?]") + 1
    df_fast["capital_count"] = text_series.str.count("[A-Z]")
    df_fast["punct_count"] = text_series.str.count('[!@#$%^&*(),.?":{}|<>]')
    df_fast["digit_count"] = text_series.str.count("\d")

    # BATCH 2: Ratios (vectorized division)
    text_len_safe = df_fast["text_length"] + 1  # Avoid division by zero
    df_fast["capital_ratio"] = df_fast["capital_count"] / text_len_safe
    df_fast["punct_ratio"] = df_fast["punct_count"] / text_len_safe
    df_fast["digit_ratio"] = df_fast["digit_count"] / text_len_safe

    # BATCH 3: Pre-compiled regex patterns for maximum speed
    print("     🔍 Pattern detection...")
    text_lower = text_series.str.lower()  # Convert once

    # Compile patterns once for massive speed improvement
    phone_pattern = re.compile(r"\d{3}[-.]?\d{3}[-.]?\d{4}", re.IGNORECASE)
    email_pattern = re.compile(r"@.*\.", re.IGNORECASE)
    url_pattern = re.compile(r"http|www\.", re.IGNORECASE)

    # Use compiled patterns
    df_fast["has_phone"] = text_series.str.contains(phone_pattern, na=False).astype(int)
    df_fast["has_email"] = text_lower.str.contains(email_pattern, na=False).astype(int)
    df_fast["has_url"] = text_lower.str.contains(url_pattern, na=False).astype(int)

    # Combined pattern matching for marketing keywords (single operation)
    marketing_pattern = re.compile(
        r"\b(sale|discount|promo|deal|offer|free|limited|visit|website)\b",
        re.IGNORECASE,
    )
    df_fast["has_marketing"] = text_lower.str.contains(
        marketing_pattern, na=False
    ).astype(int)

    # Emotional keywords (combined patterns)
    positive_pattern = re.compile(
        r"\b(amazing|awesome|best|perfect|excellent|fantastic|love)\b", re.IGNORECASE
    )
    negative_pattern = re.compile(
        r"\b(terrible|worst|awful|horrible|hate|avoid|disgusting)\b", re.IGNORECASE
    )
    df_fast["has_positive"] = text_lower.str.contains(
        positive_pattern, na=False
    ).astype(int)
    df_fast["has_negative"] = text_lower.str.contains(
        negative_pattern, na=False
    ).astype(int)

    # BATCH 4: Temporal features (vectorized datetime operations)
    print("     ⏰ Temporal features...")
    df_fast["hour"] = df_fast[time_col].dt.hour.fillna(12)
    df_fast["day_of_week"] = df_fast[time_col].dt.dayofweek.fillna(3)
    df_fast["is_weekend"] = (df_fast["day_of_week"] >= 5).astype(int)
    df_fast["is_late_night"] = (
        (df_fast["hour"] >= 23) | (df_fast["hour"] <= 5)
    ).astype(int)

    # BATCH 5: Business features (vectorized comparisons)
    print("     🏢 Business features...")
    df_fast["is_high_rated"] = (df_fast["avg_rating"] >= 4.5).astype(int)
    df_fast["is_low_rated"] = (df_fast["avg_rating"] <= 3.0).astype(int)
    df_fast["is_new_business"] = (df_fast["num_of_reviews"] <= 10).astype(int)
    df_fast["is_popular"] = (df_fast["num_of_reviews"] >= 100).astype(int)

    # Only add price level features if the column exists
    if "price_level" in df_fast.columns:
        df_fast["is_expensive"] = (df_fast["price_level"] >= 3).astype(int)
    else:
        df_fast["is_expensive"] = 0  # Default to not expensive if no price info

    # BATCH 6: Rating features (vectorized operations)
    df_fast["rating_deviation"] = df_fast["rating"] - df_fast["avg_rating"]
    df_fast["is_extreme_rating"] = (
        (df_fast["rating"] == 1) | (df_fast["rating"] == 5)
    ).astype(int)

    # BATCH 7: Simplified user features (efficient operations)
    print("     👤 User behavior features...")
    user_counts = df_fast["user_id"].value_counts()
    df_fast["user_review_count"] = df_fast["user_id"].map(user_counts).fillna(1)
    df_fast["is_prolific_user"] = (df_fast["user_review_count"] > 5).astype(int)

    # BATCH 8: Text quality indicators
    df_fast["is_very_short"] = (df_fast["word_count"] <= 3).astype(int)
    df_fast["is_very_long"] = (df_fast["word_count"] >= 50).astype(int)
    df_fast["has_excessive_caps"] = (df_fast["capital_ratio"] > 0.3).astype(int)
    df_fast["has_excessive_punct"] = (df_fast["punct_ratio"] > 0.15).astype(int)

    # BATCH 9: Interaction features
    df_fast["low_rating_positive"] = (
        (df_fast["rating"] <= 2) & (df_fast["has_positive"] == 1)
    ).astype(int)
    df_fast["high_rating_negative"] = (
        (df_fast["rating"] >= 4) & (df_fast["has_negative"] == 1)
    ).astype(int)

    # BATCH 10: Confidence score features and interactions
    print("     🎯 Confidence score features...")

    # Define confidence columns
    confidence_cols = [
        "advertisement_confidence_score",
        "irrelevant_confidence_score",
        "fake_rant_confidence_score",
    ]

    # Fill missing confidence scores with 0.5 (neutral)
    for conf_col in confidence_cols:
        if conf_col in df_fast.columns:
            df_fast[conf_col] = df_fast[conf_col].fillna(0.5)

    # Basic confidence features
    if "advertisement_confidence_score" in df_fast.columns:
        df_fast["high_ad_confidence"] = (
            df_fast["advertisement_confidence_score"] >= 0.8
        ).astype(int)
        df_fast["low_ad_confidence"] = (
            df_fast["advertisement_confidence_score"] <= 0.2
        ).astype(int)
        df_fast["moderate_ad_confidence"] = (
            (df_fast["advertisement_confidence_score"] > 0.3)
            & (df_fast["advertisement_confidence_score"] < 0.7)
        ).astype(int)

    if "irrelevant_confidence_score" in df_fast.columns:
        df_fast["high_irrelevant_confidence"] = (
            df_fast["irrelevant_confidence_score"] >= 0.8
        ).astype(int)
        df_fast["low_irrelevant_confidence"] = (
            df_fast["irrelevant_confidence_score"] <= 0.2
        ).astype(int)
        df_fast["moderate_irrelevant_confidence"] = (
            (df_fast["irrelevant_confidence_score"] > 0.3)
            & (df_fast["irrelevant_confidence_score"] < 0.7)
        ).astype(int)

    if "fake_rant_confidence_score" in df_fast.columns:
        df_fast["high_fake_confidence"] = (
            df_fast["fake_rant_confidence_score"] >= 0.8
        ).astype(int)
        df_fast["low_fake_confidence"] = (
            df_fast["fake_rant_confidence_score"] <= 0.2
        ).astype(int)
        df_fast["moderate_fake_confidence"] = (
            (df_fast["fake_rant_confidence_score"] > 0.3)
            & (df_fast["fake_rant_confidence_score"] < 0.7)
        ).astype(int)

    # Confidence score aggregations (FIXED: ensure scalar values)
    available_conf_cols = [col for col in confidence_cols if col in df_fast.columns]
    if available_conf_cols:
        # Use pandas operations to ensure scalar results
        conf_df = df_fast[available_conf_cols]
        df_fast["avg_confidence"] = conf_df.mean(axis=1).astype(float)
        df_fast["max_confidence"] = conf_df.max(axis=1).astype(float)
        df_fast["min_confidence"] = conf_df.min(axis=1).astype(float)
        df_fast["confidence_range"] = (
            df_fast["max_confidence"] - df_fast["min_confidence"]
        ).astype(float)
        df_fast["confidence_std"] = conf_df.std(axis=1).fillna(0).astype(float)

        # High confidence indicator (all scores > 0.7)
        df_fast["all_high_confidence"] = ((conf_df > 0.7).all(axis=1)).astype(int)
        # Low confidence indicator (all scores < 0.3)
        df_fast["all_low_confidence"] = ((conf_df < 0.3).all(axis=1)).astype(int)
        # Mixed confidence (high variance in scores)
        df_fast["mixed_confidence"] = (df_fast["confidence_range"] > 0.4).astype(int)

    # BATCH 11: Confidence-Rating interactions
    print("     🤝 Confidence-rating interactions...")
    if "advertisement_confidence_score" in df_fast.columns:
        # High ad confidence with low rating (suspicious)
        df_fast["high_ad_conf_low_rating"] = (
            (df_fast["high_ad_confidence"] == 1) & (df_fast["rating"] <= 2)
        ).astype(int)
        # Marketing keywords with high ad confidence
        df_fast["marketing_high_ad_conf"] = (
            (df_fast["has_marketing"] == 1) & (df_fast["high_ad_confidence"] == 1)
        ).astype(int)

    if "irrelevant_confidence_score" in df_fast.columns:
        # High irrelevant confidence with extreme ratings
        df_fast["high_irrelevant_extreme_rating"] = (
            (df_fast["high_irrelevant_confidence"] == 1)
            & (df_fast["is_extreme_rating"] == 1)
        ).astype(int)
        # Very short text with high irrelevant confidence
        df_fast["short_text_high_irrelevant"] = (
            (df_fast["is_very_short"] == 1)
            & (df_fast["high_irrelevant_confidence"] == 1)
        ).astype(int)

    if "fake_rant_confidence_score" in df_fast.columns:
        # High fake confidence with emotional words
        df_fast["high_fake_negative_words"] = (
            (df_fast["high_fake_confidence"] == 1) & (df_fast["has_negative"] == 1)
        ).astype(int)
        # Fake rant with low business rating
        df_fast["fake_rant_low_biz_rating"] = (
            (df_fast["high_fake_confidence"] == 1) & (df_fast["is_low_rated"] == 1)
        ).astype(int)

    # BATCH 12: Multi-confidence interactions
    print("     📊 Multi-confidence interactions...")
    if len(available_conf_cols) >= 2:
        # High confidence in multiple categories (suspicious)
        multi_high_count = 0
        if "advertisement_confidence_score" in df_fast.columns:
            multi_high_count += df_fast["high_ad_confidence"]
        if "irrelevant_confidence_score" in df_fast.columns:
            multi_high_count += df_fast["high_irrelevant_confidence"]
        if "fake_rant_confidence_score" in df_fast.columns:
            multi_high_count += df_fast["high_fake_confidence"]

        df_fast["multi_high_confidence"] = (multi_high_count >= 2).astype(int)

        # Confidence consistency with text features
        df_fast["confidence_text_mismatch"] = (
            (df_fast["mixed_confidence"] == 1)
            & ((df_fast["has_marketing"] == 1) | (df_fast["has_negative"] == 1))
        ).astype(int)

    # BATCH 13: Confidence-Business interactions
    print("     🏢 Confidence-business interactions...")
    if "advertisement_confidence_score" in df_fast.columns:
        # New business with high ad confidence
        df_fast["new_biz_high_ad_conf"] = (
            (df_fast["is_new_business"] == 1) & (df_fast["high_ad_confidence"] == 1)
        ).astype(int)

    if "fake_rant_confidence_score" in df_fast.columns:
        # Popular business with fake rants
        df_fast["popular_biz_fake_rant"] = (
            (df_fast["is_popular"] == 1) & (df_fast["high_fake_confidence"] == 1)
        ).astype(int)

    # BATCH 14: Confidence-User interactions
    print("     👤 Confidence-user interactions...")
    if len(available_conf_cols) > 0:
        # Prolific users with high confidence scores
        df_fast["prolific_user_high_conf"] = (
            (df_fast["is_prolific_user"] == 1) & (df_fast["avg_confidence"] >= 0.7)
        ).astype(int)
        # Late night reviews with high confidence
        df_fast["late_night_high_conf"] = (
            (df_fast["is_late_night"] == 1) & (df_fast["max_confidence"] >= 0.8)
        ).astype(int)

    return df_fast

In [ ]:
# 🔍 CONFIDENCE SCORE VALIDATION & FEATURE PREVIEW

print("🔍 VALIDATING CONFIDENCE SCORE FEATURES...")

# Check if confidence score columns exist
confidence_cols = [
    "advertisement_confidence_score",
    "irrelevant_confidence_score",
    "fake_rant_confidence_score",
]
existing_conf_cols = [col for col in confidence_cols if col in reviews_df.columns]

print(f"📊 Available confidence columns: {existing_conf_cols}")

if existing_conf_cols:
    print(f"\n📈 Confidence Score Statistics:")
    for col in existing_conf_cols:
        stats = reviews_df[col].describe()
        print(f"   {col}:")
        print(f"      Range: {stats['min']:.3f} - {stats['max']:.3f}")
        print(f"      Mean: {stats['mean']:.3f} ± {stats['std']:.3f}")
        print(f"      Missing: {reviews_df[col].isnull().sum()}")

    # Preview confidence distributions
    print(f"\n🎯 Confidence Score Value Counts (sample):")
    for col in existing_conf_cols[:2]:  # Show first 2 to save space
        print(f"\n   {col}:")
        value_counts = reviews_df[col].value_counts().head(10)
        for val, count in value_counts.items():
            print(f"      {val:.3f}: {count}")
else:
    print(
        "⚠️  No confidence score columns found. Features will be created but won't include confidence interactions."
    )

print(f"\n✅ Ready to proceed with enhanced feature engineering!")

In [ ]:
# 📊 ENHANCED FEATURE ENGINEERING PERFORMANCE MONITORING

print("📊 PERFORMANCE MONITORING FOR ENHANCED FEATURES...")

# Apply ultra-fast feature engineering
print(f"\n🔧 APPLYING ENHANCED ULTRA-FAST PIPELINE...")
feature_start_time = time.time()

df_engineered = ultra_fast_feature_engineering(reviews_df)

feature_elapsed_time = time.time() - feature_start_time

# Get engineered feature names
all_columns = set(df_engineered.columns)
used_columns = set(
    TARGET_COLUMNS
    + TEXT_COLUMNS
    + [
        "user_id",
        "name_x",  # Fixed column name
        "time",  # Fixed column name
        "gmap_id",
        "has_pics",  # Fixed column name
        "name_y",  # Fixed column name
        "avg_rating",
        "num_of_reviews",
        "latitude",
        "longitude",
        "state",
        "token_count",
        "pics",
        "resp",
    ]
)
ENGINEERED_METADATA_COLUMNS = sorted(list(all_columns - used_columns))

print(f"\n✅ ENHANCED FEATURE ENGINEERING COMPLETE!")
print(f"   ⚡ Processing time: {feature_elapsed_time:.2f} seconds")
print(f"   📊 Original features: {len(reviews_df.columns)}")
print(f"   🚀 Total features: {len(df_engineered.columns)}")
print(f"   📈 New engineered features: {len(ENGINEERED_METADATA_COLUMNS)}")
print(f"   🎯 Speed: {len(reviews_df)/feature_elapsed_time:.0f} rows/second")

# Count confidence-related features
confidence_features = [
    f for f in ENGINEERED_METADATA_COLUMNS if "confidence" in f.lower()
]
print(f"   🎯 Confidence-based features: {len(confidence_features)}")

print(f"\n📋 CONFIDENCE-RELATED FEATURES ({len(confidence_features)}):")
for i, feature in enumerate(confidence_features):
    if i % 4 == 0:
        print()
    print(f"   {feature:<25}", end="")

print(f"\n\n📋 ALL ENGINEERED FEATURES ({len(ENGINEERED_METADATA_COLUMNS)}):")
for i, feature in enumerate(ENGINEERED_METADATA_COLUMNS):
    if i % 6 == 0:
        print()
    print(f"   {feature:<20}", end="")

print(f"\n\n🎉 READY FOR ENHANCED XGBOOST TRAINING!")

In [ ]:
# ✅ ENHANCED FEATURE VALIDATION & DATA INTEGRITY CHECK

print("✅ VALIDATING ENHANCED FEATURES & DATA INTEGRITY...")

# Check for missing values in key columns
print(f"\n📊 DATA INTEGRITY CHECK:")
print(f"   Total rows: {len(df_engineered):,}")
print(f"   Total columns: {len(df_engineered.columns)}")

# Check confidence-related features
confidence_features = [
    f for f in ENGINEERED_METADATA_COLUMNS if "confidence" in f.lower()
]
if confidence_features:
    print(f"\n🎯 CONFIDENCE FEATURE VALIDATION:")
    sample_conf_features = confidence_features[:5]  # Show first 5
    for feature in sample_conf_features:
        if feature in df_engineered.columns:
            non_null_count = df_engineered[feature].count()
            null_count = df_engineered[feature].isnull().sum()
            print(f"   {feature}: {non_null_count:,} valid, {null_count} missing")

            # Show data type and sample values
            dtype = df_engineered[feature].dtype
            unique_vals = df_engineered[feature].nunique()
            print(f"      Type: {dtype}, Unique values: {unique_vals}")

            # Show sample values for binary features
            if unique_vals <= 10:
                sample_vals = df_engineered[feature].value_counts().head(3)
                print(f"      Sample values: {dict(sample_vals)}")

# Verify no duplicate or problematic columns
duplicated_cols = df_engineered.columns[df_engineered.columns.duplicated()].tolist()
if duplicated_cols:
    print(f"⚠️  Duplicate columns found: {duplicated_cols}")
else:
    print(f"✅ No duplicate columns found")

# Check memory usage
memory_usage = df_engineered.memory_usage(deep=True).sum() / (1024**2)  # MB
print(f"\n💾 Memory usage: {memory_usage:.2f} MB")

# Verify target columns exist
missing_targets = [col for col in TARGET_COLUMNS if col not in df_engineered.columns]
if missing_targets:
    print(f"❌ Missing target columns: {missing_targets}")
else:
    print(f"✅ All target columns present: {TARGET_COLUMNS}")

print(f"\n🎉 ENHANCED FEATURES VALIDATED! Ready for model training.")
print(f"   🚀 Total engineered features: {len(ENGINEERED_METADATA_COLUMNS)}")
print(f"   🎯 Confidence-based features: {len(confidence_features)}")
print(
    f"   📊 Feature engineering speed: {len(reviews_df)/feature_elapsed_time:.0f} rows/sec"
)

## 🎯 Enhanced Confidence-Based Feature Engineering Summary

### 🚀 **New Confidence Score Features Added:**

#### **📊 BATCH 10: Basic Confidence Features**

- `high_ad_confidence`, `low_ad_confidence`, `moderate_ad_confidence`
- `high_irrelevant_confidence`, `low_irrelevant_confidence`, `moderate_irrelevant_confidence`
- `high_fake_confidence`, `low_fake_confidence`, `moderate_fake_confidence`
- `avg_confidence`, `max_confidence`, `min_confidence`, `confidence_range`, `confidence_std`
- `all_high_confidence`, `all_low_confidence`, `mixed_confidence`

#### **🤝 BATCH 11: Confidence-Rating Interactions**

- `high_ad_conf_low_rating` - High advertisement confidence with low ratings (suspicious)
- `marketing_high_ad_conf` - Marketing keywords combined with high ad confidence
- `high_irrelevant_extreme_rating` - High irrelevant confidence with extreme ratings
- `short_text_high_irrelevant` - Very short text with high irrelevant confidence
- `high_fake_negative_words` - High fake confidence with negative emotional words
- `fake_rant_low_biz_rating` - Fake rant confidence with low-rated businesses

#### **📊 BATCH 12: Multi-Confidence Interactions**

- `multi_high_confidence` - High confidence in multiple categories (suspicious)
- `confidence_text_mismatch` - Mixed confidence with inconsistent text features

#### **🏢 BATCH 13: Confidence-Business Interactions**

- `new_biz_high_ad_conf` - New businesses with high advertisement confidence
- `popular_biz_fake_rant` - Popular businesses targeted by fake rants

#### **👤 BATCH 14: Confidence-User Interactions**

- `prolific_user_high_conf` - Prolific users with high confidence scores
- `late_night_high_conf` - Late night reviews with high confidence scores

### ⚡ **Performance Improvements:**

- **Lightning-fast vectorized operations** for all confidence calculations
- **Smart missing value handling** with 0.5 default (neutral confidence)
- **Batch processing** of multiple confidence scores simultaneously
- **Memory-efficient** numpy array operations for aggregations

### 🎯 **Expected XGBoost Benefits:**

- **Better spam detection** through confidence-rating mismatches
- **Enhanced advertisement detection** via marketing keyword + confidence interactions
- **Improved fake review identification** using multi-confidence patterns
- **Sophisticated user behavior modeling** with confidence-user interactions
- **Business context awareness** through confidence-business feature combinations


## 3. 📈 Baseline Multi-Label XGBoost Model

Let's train a baseline multi-output XGBoost model and evaluate its performance.


In [ ]:
# Create the main preprocessor
metadata_preprocessor = ColumnTransformer(
    transformers=preprocessor_components, remainder="drop"  # Removed n_jobs=-1
)

print(f"\n🔧 PREPROCESSING PIPELINE CREATED:")
print(f"   ✅ Numerical features: Median imputation + StandardScaler")
print(f"   ✅ Categorical features: Mode imputation + OneHotEncoder")
print(f"   ✅ Binary features: Zero imputation")

print(f"\n🚀 APPLYING PREPROCESSING TO {len(metadata_cols)} ENGINEERED FEATURES...")

# Use only valid columns (not arrays)
valid_cols = numerical_features + categorical_features + binary_features
print(f"   📊 Using {len(valid_cols)} valid columns for preprocessing")

if valid_cols:
    # Apply preprocessing to metadata features
    metadata_processed = metadata_preprocessor.fit_transform(metadata_df[valid_cols])

    print(f"✅ Metadata preprocessing complete: {metadata_processed.shape}")
    print(f"   📊 Original metadata features: {len(metadata_cols)}")
    print(f"   🔄 Processed feature dimensions: {metadata_processed.shape[1]}")
else:
    print("❌ No valid columns found for preprocessing")

elapsed_time = time.time() - start_time
print(f"\n⚡ Preprocessing completed in {elapsed_time:.2f} seconds")
print(f"🎯 Ready for text embeddings and model training!")

In [ ]:
# 🚀 FAST XGBOOST TRAINING WITH COMBINED FEATURES

import time
from sklearn.model_selection import train_test_split
import xgboost as xgb

start_time = time.time()
print("🎯 CREATING COMBINED FEATURE MATRIX...")

# ========================================
# STEP 1: COMBINE ALL FEATURES
# ========================================

print(f"📊 COMBINING TEXT EMBEDDINGS + METADATA FEATURES...")

# Combine text embeddings + processed metadata
X_final = np.hstack([combined_embeddings, metadata_processed])
y_final = df_engineered[TARGET_COLUMNS].values

print(f"✅ FEATURE COMBINATION COMPLETE:")
print(f"   📊 Text embeddings: {combined_embeddings.shape[1]} features")
print(f"   🔧 Metadata features: {metadata_processed.shape[1]} features")
print(f"   🎯 Total features: {X_final.shape[1]}")
print(f"   📈 Samples: {X_final.shape[0]}")
print(f"   🎯 Targets: {y_final.shape[1]} ({TARGET_COLUMNS})")

# ========================================
# STEP 2: FAST DATA SPLITTING
# ========================================

print(f"\n📊 CREATING DATA SPLITS...")

# Simple stratified split on first target (fastest approach)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_final,
    y_final,
    test_size=0.4,
    random_state=42,
    stratify=y_final[:, 0],  # Stratify on first target only for speed
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp[:, 0],  # Stratify on first target only for speed
)

print(f"✅ DATA SPLIT SUMMARY:")
print(f"   🏋️  Train set: {X_train.shape[0]} samples (60%)")
print(f"   🔍 Validation set: {X_val.shape[0]} samples (20%)")
print(f"   🧪 Test set: {X_test.shape[0]} samples (20%)")

# ========================================
# STEP 3: FAST XGBOOST TRAINING
# ========================================

print(f"\n🚀 TRAINING FAST XGBOOST MODELS...")

# Train individual optimized models for each target
individual_models = {}
training_results = {}

# Fast XGBoost configuration
xgb_config = {
    "n_estimators": 100,  # Reduced for speed
    "max_depth": 6,
    "learning_rate": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "tree_method": "hist",  # Fastest method
    "random_state": 42,
    "eval_metric": "logloss",
    "n_jobs": -1,  # Use all cores
    "verbosity": 0,  # Silent
}

for i, target in enumerate(TARGET_COLUMNS):
    print(f"   🎯 Training {target}...")

    # Calculate class balance weight
    pos_rate = y_train[:, i].mean()
    scale_pos_weight = (1 - pos_rate) / pos_rate if pos_rate > 0 else 1.0

    # Create model with class balance
    config = xgb_config.copy()
    config["scale_pos_weight"] = scale_pos_weight

    model = xgb.XGBClassifier(**config)

    # Fast training without early stopping
    model.fit(X_train, y_train[:, i])

    individual_models[target] = model

    # Quick validation
    val_pred = model.predict(X_val)
    val_f1 = f1_score(y_val[:, i], val_pred, zero_division=0)

    training_results[target] = {"scale_pos_weight": scale_pos_weight, "val_f1": val_f1}

    print(f"      ✅ Validation F1: {val_f1:.4f} | Weight: {scale_pos_weight:.2f}")

# ========================================
# STEP 4: GENERATE PREDICTIONS
# ========================================

print(f"\n📊 GENERATING PREDICTIONS...")

# Generate predictions for all sets
y_train_pred = np.column_stack(
    [individual_models[target].predict(X_train) for target in TARGET_COLUMNS]
)
y_val_pred = np.column_stack(
    [individual_models[target].predict(X_val) for target in TARGET_COLUMNS]
)

y_train_proba = [
    individual_models[target].predict_proba(X_train) for target in TARGET_COLUMNS
]
y_val_proba = [
    individual_models[target].predict_proba(X_val) for target in TARGET_COLUMNS
]

elapsed_time = time.time() - start_time
print(f"\n✅ FAST XGBOOST TRAINING COMPLETE!")
print(f"   ⚡ Training time: {elapsed_time:.2f} seconds")
print(f"   🎯 Trained {len(individual_models)} models")
print(f"   📊 Total features used: {X_final.shape[1]}")
print(f"   🎯 Speed: {len(TARGET_COLUMNS)/(elapsed_time):.1f} models/second")

# Quick performance summary
print(f"\n📈 QUICK PERFORMANCE SUMMARY:")
for target in TARGET_COLUMNS:
    val_f1 = training_results[target]["val_f1"]
    print(f"   🎯 {target}: F1 = {val_f1:.4f}")

print(f"\n🎉 READY FOR DETAILED EVALUATION!")

In [ ]:
# Baseline model evaluation
from sklearn.metrics import (
    classification_report,
    precision_recall_curve,
    average_precision_score,
)
from sklearn.metrics import roc_auc_score, roc_curve

target_cols = ["advertisement", "irrelevant", "fake_rant"]


def evaluate_multilabel_model(y_true, y_pred, y_proba, target_names, dataset_name=""):
    """Comprehensive evaluation of multi-label model"""

    print(f"\n📊 {dataset_name} Performance Metrics:")
    print("=" * 60)

    # Overall metrics
    metrics_summary = []

    for i, target in enumerate(target_names):
        # Extract single-label predictions and probabilities
        y_true_single = y_true[:, i]
        y_pred_single = y_pred[:, i]
        y_proba_single = (
            y_proba[i][:, 1]
            if hasattr(y_proba[i], "shape") and y_proba[i].shape[1] > 1
            else y_proba[i]
        )

        # Calculate metrics
        precision = precision_score(y_true_single, y_pred_single, zero_division=0)
        recall = recall_score(y_true_single, y_pred_single, zero_division=0)
        f1 = f1_score(y_true_single, y_pred_single, zero_division=0)
        accuracy = accuracy_score(y_true_single, y_pred_single)

        metrics_summary.append(
            {
                "Target": target,
                "Precision": precision,
                "Recall": recall,
                "F1-Score": f1,
                "Accuracy": accuracy,
            }
        )

        print(f"\n🎯 {target}:")
        print(f"   Precision: {precision:.4f}")
        print(f"   Recall: {recall:.4f}")
        print(f"   F1-Score: {f1:.4f}")
        print(f"   Accuracy: {accuracy:.4f}")
        print(f"   Support: {y_true_single.sum()}")

        # Classification report for this target
        print(f"\n   Detailed Report:")
        print(
            classification_report(
                y_true_single,
                y_pred_single,
                labels=[0, 1],  # force both classes
                target_names=["Negative", "Positive"],
                digits=4,
                zero_division=0,  # avoid divide-by-zero warnings
            )
        )

    return metrics_summary


# Evaluate baseline model
print("🔍 Evaluating Baseline Multi-Output XGBoost Model")

train_metrics = evaluate_multilabel_model(
    y_train, y_train_pred, y_train_proba, target_cols, "TRAINING SET"
)

val_metrics = evaluate_multilabel_model(
    y_val, y_val_pred, y_val_proba, target_cols, "VALIDATION SET"
)

# Create summary DataFrame
train_df = pd.DataFrame(train_metrics)
val_df = pd.DataFrame(val_metrics)

print(f"\n📈 Performance Summary:")
print(f"\nTRAINING SET:")
print(train_df.round(4).to_string(index=False))
print(f"\nVALIDATION SET:")
print(val_df.round(4).to_string(index=False))

In [ ]:
# Visualization functions for comprehensive model evaluation


def plot_confusion_matrices(y_true, y_pred, target_names, title_prefix=""):
    """Plot confusion matrices for all targets"""
    n_targets = len(target_names)
    fig, axes = plt.subplots(1, n_targets, figsize=(5 * n_targets, 4))
    if n_targets == 1:
        axes = [axes]

    for i, (target, ax) in enumerate(zip(target_names, axes)):
        cm = confusion_matrix(y_true[:, i], y_pred[:, i])

        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            ax=ax,
            xticklabels=["Negative", "Positive"],
            yticklabels=["Negative", "Positive"],
        )
        ax.set_title(f"{title_prefix}{target}")
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")

    plt.tight_layout()
    plt.show()


def plot_precision_recall_curves(y_true, y_proba, target_names, title_prefix=""):
    """Plot precision-recall curves for all targets"""
    n_targets = len(target_names)
    fig, axes = plt.subplots(1, n_targets, figsize=(5 * n_targets, 4))
    if n_targets == 1:
        axes = [axes]

    for i, (target, ax) in enumerate(zip(target_names, axes)):
        y_true_single = y_true[:, i]
        y_proba_single = (
            y_proba[i][:, 1]
            if hasattr(y_proba[i], "shape") and y_proba[i].shape[1] > 1
            else y_proba[i]
        )

        precision, recall, _ = precision_recall_curve(y_true_single, y_proba_single)
        ap_score = average_precision_score(y_true_single, y_proba_single)

        ax.plot(recall, precision, label=f"AP = {ap_score:.3f}")
        ax.fill_between(recall, precision, alpha=0.3)
        ax.set_xlabel("Recall")
        ax.set_ylabel("Precision")
        ax.set_title(f"{title_prefix}{target}")
        ax.legend()
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


def plot_f1_threshold_curves(y_true, y_proba, target_names, title_prefix=""):
    """Plot F1 score vs threshold curves"""
    n_targets = len(target_names)
    fig, axes = plt.subplots(1, n_targets, figsize=(5 * n_targets, 4))
    if n_targets == 1:
        axes = [axes]

    for i, (target, ax) in enumerate(zip(target_names, axes)):
        y_true_single = y_true[:, i]
        y_proba_single = (
            y_proba[i][:, 1]
            if hasattr(y_proba[i], "shape") and y_proba[i].shape[1] > 1
            else y_proba[i]
        )

        # Calculate F1 scores for different thresholds
        thresholds = np.linspace(0.1, 0.9, 50)
        f1_scores = []

        for threshold in thresholds:
            y_pred_thresh = (y_proba_single >= threshold).astype(int)
            f1 = f1_score(y_true_single, y_pred_thresh, zero_division=0)
            f1_scores.append(f1)

        best_idx = np.argmax(f1_scores)
        best_threshold = thresholds[best_idx]
        best_f1 = f1_scores[best_idx]

        ax.plot(thresholds, f1_scores, "b-", linewidth=2)
        ax.axvline(x=best_threshold, color="r", linestyle="--", alpha=0.7)
        ax.axhline(y=best_f1, color="r", linestyle="--", alpha=0.7)
        ax.scatter([best_threshold], [best_f1], color="red", s=100, zorder=5)

        ax.set_xlabel("Threshold")
        ax.set_ylabel("F1 Score")
        ax.set_title(
            f"{title_prefix}{target}\nBest F1: {best_f1:.3f} @ {best_threshold:.3f}"
        )
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


# Plot baseline model visualizations
print("📊 Generating Baseline Model Visualizations...")

# Confusion matrices
plot_confusion_matrices(y_val, y_val_pred, target_cols, "Baseline Val - ")

# Precision-Recall curves
plot_precision_recall_curves(y_val, y_val_proba, target_cols, "Baseline Val - ")

# F1 vs Threshold curves
plot_f1_threshold_curves(y_val, y_val_proba, target_cols, "Baseline Val - ")

In [ ]:
# Analysis and Learning Curves
from sklearn.model_selection import learning_curve


def plot_learning_curves(estimator, X, y, target_names, title_prefix=""):
    """Plot learning curves for multi-output targets"""
    n_targets = len(target_names)
    fig, axes = plt.subplots(1, n_targets, figsize=(5 * n_targets, 4))

    if n_targets == 1:
        axes = [axes]

    for i, (target, ax) in enumerate(zip(target_names, axes)):
        # Create single-output classifier for this target
        single_clf = xgb.XGBClassifier(
            n_estimators=100,
            max_depth=6,
            tree_method="hist",
            random_state=42,
        )

        train_sizes, train_scores, val_scores = learning_curve(
            single_clf,
            X,
            y[:, i],
            cv=3,
            train_sizes=np.linspace(0.1, 1.0, 10),
            scoring="f1",
        )

        train_mean = np.mean(train_scores, axis=1)
        train_std = np.std(train_scores, axis=1)
        val_mean = np.mean(val_scores, axis=1)
        val_std = np.std(val_scores, axis=1)

        ax.plot(train_sizes, train_mean, "o-", color="blue", label="Training F1")
        ax.fill_between(
            train_sizes,
            train_mean - train_std,
            train_mean + train_std,
            alpha=0.1,
            color="blue",
        )

        ax.plot(train_sizes, val_mean, "o-", color="red", label="Validation F1")
        ax.fill_between(
            train_sizes,
            val_mean - val_std,
            val_mean + val_std,
            alpha=0.1,
            color="red",
        )

        ax.set_xlabel("Training Set Size")
        ax.set_ylabel("F1 Score")
        ax.set_title(f"{title_prefix}{target}")
        ax.legend()
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


# 🔍 Analyzing Model Performance and Potential Issues...
print("🔍 Analyzing Model Performance and Potential Issues...")

print(f"\n📊 Overfitting Analysis:")

# Check for overfitting by comparing training vs validation scores
for i, target in enumerate(target_cols):
    train_f1 = f1_score(y_train[:, i], y_train_pred[:, i])
    val_f1 = f1_score(y_val[:, i], y_val_pred[:, i])
    gap = train_f1 - val_f1

    print(f"\n🎯 {target}:")
    print(f"   Training F1: {train_f1:.4f}")
    print(f"   Validation F1: {val_f1:.4f}")
    print(f"   Gap: {gap:.4f}")

    if gap > 0.1:
        print(f"   ⚠️  Potential overfitting detected (gap > 0.1)")
    else:
        print(f"   ⚠️  Moderate overfitting")

# Plot learning curves
print(f"\n📈 Generating Learning Curves...")
plot_learning_curves(model, X_train, y_train, target_cols, "Learning Curve - ")

# Analyze class imbalance and potential solutions
print(f"\n⚖️  Class Imbalance Analysis:")
for i, target in enumerate(target_cols):
    pos_rate = y_train[:, i].mean()
    imbalance_ratio = (1 - pos_rate) / pos_rate if pos_rate > 0 else float("inf")

    print(f"\n🎯 {target}:")
    print(f"   Positive rate: {pos_rate:.3f}")
    print(f"   Imbalance ratio: {imbalance_ratio:.2f}:1")

    if imbalance_ratio > 10:
        print(f"   ⚠️  Severe class imbalance - consider scale_pos_weight")
    elif imbalance_ratio > 5:
        print(f"   ⚠️  Moderate class imbalance - may need adjustment")
    else:
        print(f"   ✅ Reasonable class balance")

print(f"\n🎯 Recommendations for Hyperparameter Optimization:")
print(f"   • Consider scale_pos_weight for class imbalance")
print(f"   • Tune n_estimators (try 200-500)")
print(f"   • Adjust max_depth (try 4-8)")
print(f"   • Modify learning_rate (try 0.05-0.2)")
print(f"   • Use colsample_bytree (try 0.8-1.0)")
print(f"   • Add regularization with gamma (try 0-2)")
print(f"   • Optimize thresholds for better F1 scores")

## 4. 🔧 Hyperparameter Optimization with RandomizedSearchCV

Based on our baseline analysis, let's optimize hyperparameters to improve performance.


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.metrics import make_scorer, f1_score

# Define comprehensive hyperparameter search space
param_distributions = {
    "estimator__n_estimators": randint(100, 500),
    "estimator__max_depth": randint(3, 10),
    "estimator__learning_rate": uniform(0.05, 0.15),
    "estimator__subsample": uniform(0.7, 0.3),
    "estimator__colsample_bytree": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 2),
    "estimator__reg_alpha": uniform(0, 1),
    "estimator__reg_lambda": uniform(1, 2),
    "estimator__min_child_weight": randint(1, 6),
}

# Calculate scale_pos_weight for each target (to handle class imbalance)
scale_pos_weights = []
for i in range(len(target_cols)):
    pos_rate = y_train[:, i].mean()
    if pos_rate > 0:
        scale_pos_weight = (1 - pos_rate) / pos_rate
    else:
        scale_pos_weight = 1.0
    scale_pos_weights.append(scale_pos_weight)
    print(f"📊 {target_cols[i]} scale_pos_weight: {scale_pos_weight:.2f}")


# Custom scorer that averages F1 scores across all targets
def multi_target_f1_scorer(y_true, y_pred):
    """Calculate average F1 score across all targets"""
    f1_scores = []
    for i in range(y_true.shape[1]):
        f1 = f1_score(y_true[:, i], y_pred[:, i], zero_division=0)
        f1_scores.append(f1)
    return np.mean(f1_scores)


# Create custom scorer
multi_f1_scorer = make_scorer(multi_target_f1_scorer)

print(f"\n🔍 Starting Hyperparameter Optimization...")
print(f"   Search space size: {len(param_distributions)} parameters")
print(f"   Using RandomizedSearchCV with 25 iterations (reduced for speed)")
print(f"   Cross-validation: 3-fold")
print(f"   Scoring: F1 for each target separately")

# Perform hyperparameter search for each target separately (for better control)
best_models = {}
best_params = {}

for i, target in enumerate(target_cols):
    print(f"\n🎯 Optimizing hyperparameters for {target}...")

    # Create base classifier with target-specific scale_pos_weight
    base_clf = xgb.XGBClassifier(
        tree_method="hist",
        random_state=42,
        eval_metric="logloss",
        scale_pos_weight=scale_pos_weights[i],
    )

    # Single parameter search space (without estimator__ prefix for single target)
    single_param_dist = {
        "n_estimators": randint(100, 300),  # Reduced range for speed
        "max_depth": randint(3, 8),
        "learning_rate": uniform(0.05, 0.15),
        "subsample": uniform(0.7, 0.3),
        "colsample_bytree": uniform(0.7, 0.3),
        "gamma": uniform(0, 1),  # Reduced range
        "reg_alpha": uniform(0, 0.5),  # Reduced range
        "reg_lambda": uniform(1, 1.5),  # Reduced range
        "min_child_weight": randint(1, 4),  # Reduced range
    }

    # RandomizedSearchCV (removed n_jobs=-1)
    random_search = RandomizedSearchCV(
        estimator=base_clf,
        param_distributions=single_param_dist,
        n_iter=25,  # Reduced iterations for speed
        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
        scoring="f1",
        random_state=42,
        verbose=1,
    )

    # Fit the search
    random_search.fit(X_train, y_train[:, i])

    # Store best model and parameters
    best_models[target] = random_search.best_estimator_
    best_params[target] = random_search.best_params_

    print(f"   ✅ Best CV F1 score: {random_search.best_score_:.4f}")
    print(f"   ✅ Best parameters: {random_search.best_params_}")

print(f"\n🎉 Hyperparameter optimization complete!")

In [ ]:
# Create optimized multi-output classifier and evaluate
print(f"\n🚀 Training Optimized Multi-Output Model...")
from sklearn.multioutput import MultiOutputClassifier

# Create multi-output classifier with optimized models
optimized_xgb = MultiOutputClassifier([best_models[target] for target in target_cols])

# Since we already have trained individual models, we can use them directly
# Make predictions with optimized models
y_train_pred_opt = np.column_stack(
    [best_models[target].predict(X_train) for target in target_cols]
)
y_val_pred_opt = np.column_stack(
    [best_models[target].predict(X_val) for target in target_cols]
)
y_train_proba_opt = [
    best_models[target].predict_proba(X_train) for target in target_cols
]
y_val_proba_opt = [best_models[target].predict_proba(X_val) for target in target_cols]

print(f"✅ Optimized model predictions complete!")

# Evaluate optimized model
print(f"\n🔍 Evaluating Optimized Model...")

train_metrics_opt = evaluate_multilabel_model(
    y_train, y_train_pred_opt, y_train_proba_opt, target_cols, "OPTIMIZED TRAINING SET"
)

val_metrics_opt = evaluate_multilabel_model(
    y_val, y_val_pred_opt, y_val_proba_opt, target_cols, "OPTIMIZED VALIDATION SET"
)

# Compare baseline vs optimized performance
print(f"\n📊 PERFORMANCE COMPARISON: Baseline vs Optimized")
print("=" * 80)

comparison_data = []
for i, target in enumerate(target_cols):
    baseline_f1 = f1_score(y_val[:, i], y_val_pred[:, i])
    optimized_f1 = f1_score(y_val[:, i], y_val_pred_opt[:, i])
    improvement = optimized_f1 - baseline_f1

    if baseline_f1 > 0:
        improvement_pct = improvement / baseline_f1 * 100
    else:
        improvement_pct = 0.0  # or maybe just improvement*100 if you prefer

    comparison_data.append(
        {
            "Target": target,
            "Baseline_F1": baseline_f1,
            "Optimized_F1": optimized_f1,
            "Improvement": improvement,
            "Improvement_%": (
                (improvement / baseline_f1 * 100) if baseline_f1 > 0 else 0
            ),
        }
    )

    print(f"\n🎯 {target}:")
    print(f"   Baseline F1:  {baseline_f1:.4f}")
    print(f"   Optimized F1: {optimized_f1:.4f}")
    print(f"   Improvement:  {improvement:+.4f} ({improvement_pct:+.1f}%)")

comparison_df = pd.DataFrame(comparison_data)
print(f"\n📈 Summary Table:")
print(comparison_df.round(4).to_string(index=False))

In [ ]:
# Visualizations for optimized model
print("📊 Generating Optimized Model Visualizations...")

# Confusion matrices for optimized model
plot_confusion_matrices(y_val, y_val_pred_opt, target_cols, "Optimized Val - ")

# Precision-Recall curves for optimized model
plot_precision_recall_curves(y_val, y_val_proba_opt, target_cols, "Optimized Val - ")

# F1 vs Threshold curves for optimized model
plot_f1_threshold_curves(y_val, y_val_proba_opt, target_cols, "Optimized Val - ")

# Feature importance analysis for optimized models
print(f"\n🔍 Feature Importance Analysis (Optimized Models):")


def plot_feature_importance(models, target_names, feature_names, top_k=15):
    """Plot feature importance for optimized models"""
    n_targets = len(target_names)
    fig, axes = plt.subplots(1, n_targets, figsize=(6 * n_targets, 8))
    if n_targets == 1:
        axes = [axes]

    for i, (target, ax) in enumerate(zip(target_names, axes)):
        model = models[target]
        importances = model.feature_importances_

        # Get top features
        indices = np.argsort(importances)[-top_k:]
        top_features = [
            feature_names[idx] if idx < len(feature_names) else f"feature_{idx}"
            for idx in indices
        ]
        top_importances = importances[indices]

        # Plot horizontal bar chart
        ax.barh(range(len(top_features)), top_importances)
        ax.set_yticks(range(len(top_features)))
        ax.set_yticklabels(top_features)
        ax.set_xlabel("Feature Importance")
        ax.set_title(f"{target} - Top {top_k} Features")

        # Add value labels
        for j, v in enumerate(top_importances):
            ax.text(v + 0.001, j, f"{v:.3f}", va="center")

    plt.tight_layout()
    plt.show()


# Create comprehensive feature names list
embedding_features = [f"embed_{i}" for i in range(combined_embeddings.shape[1])]
comprehensive_feature_names = embedding_features + metadata_cols

print(f"📊 Total features: {len(comprehensive_feature_names)}")
print(f"   - Embedding features: {len(embedding_features)}")
print(f"   - Metadata features: {len(metadata_cols)}")

# Plot feature importance
plot_feature_importance(best_models, target_cols, comprehensive_feature_names, top_k=15)

# Learning curves for optimized models
print(f"\n📈 Learning Curves for Optimized Models:")
plot_learning_curves(optimized_xgb, X_train, y_train, target_cols, "Optimized - ")

## 5. 🏆 Final Model Evaluation on Test Set

Now let's evaluate our optimized model on the held-out test set.


In [ ]:
# Final evaluation on test set
print("🏆 FINAL MODEL EVALUATION ON TEST SET")
print("=" * 60)

# Make predictions on test set
y_test_pred_final = np.column_stack(
    [best_models[target].predict(X_test) for target in target_cols]
)
y_test_proba_final = [
    best_models[target].predict_proba(X_test) for target in target_cols
]

print(f"✅ Final model predictions on test set complete!")

# Evaluate final model on test set
test_metrics_final = evaluate_multilabel_model(
    y_test, y_test_pred_final, y_test_proba_final, target_cols, "FINAL TEST SET"
)

# Create comprehensive results summary
print(f"\n📊 COMPREHENSIVE RESULTS SUMMARY")
print("=" * 80)

final_results = []
for i, target in enumerate(target_cols):
    # Calculate metrics for all sets
    train_f1 = f1_score(y_train[:, i], y_train_pred_opt[:, i])
    val_f1 = f1_score(y_val[:, i], y_val_pred_opt[:, i])
    test_f1 = f1_score(y_test[:, i], y_test_pred_final[:, i])

    train_precision = precision_score(y_train[:, i], y_train_pred_opt[:, i])
    val_precision = precision_score(y_val[:, i], y_val_pred_opt[:, i])
    test_precision = precision_score(y_test[:, i], y_test_pred_final[:, i])

    train_recall = recall_score(y_train[:, i], y_train_pred_opt[:, i])
    val_recall = recall_score(y_val[:, i], y_val_pred_opt[:, i])
    test_recall = recall_score(y_test[:, i], y_test_pred_final[:, i])

    final_results.append(
        {
            "Target": target,
            "Train_F1": train_f1,
            "Val_F1": val_f1,
            "Test_F1": test_f1,
            "Train_Precision": train_precision,
            "Val_Precision": val_precision,
            "Test_Precision": test_precision,
            "Train_Recall": train_recall,
            "Val_Recall": val_recall,
            "Test_Recall": test_recall,
        }
    )

final_df = pd.DataFrame(final_results)

# Display results
print(f"\n🎯 F1 SCORES ACROSS ALL SETS:")
f1_cols = ["Target", "Train_F1", "Val_F1", "Test_F1"]
print(final_df[f1_cols].round(4).to_string(index=False))

print(f"\n🎯 PRECISION SCORES ACROSS ALL SETS:")
precision_cols = ["Target", "Train_Precision", "Val_Precision", "Test_Precision"]
print(final_df[precision_cols].round(4).to_string(index=False))

print(f"\n🎯 RECALL SCORES ACROSS ALL SETS:")
recall_cols = ["Target", "Train_Recall", "Val_Recall", "Test_Recall"]
print(final_df[recall_cols].round(4).to_string(index=False))

# Calculate overall metrics
overall_test_f1 = np.mean([final_df["Test_F1"]])
overall_test_precision = np.mean([final_df["Test_Precision"]])
overall_test_recall = np.mean([final_df["Test_Recall"]])

print(f"\n🏆 OVERALL TEST SET PERFORMANCE:")
print(f"   Average F1 Score: {overall_test_f1:.4f}")
print(f"   Average Precision: {overall_test_precision:.4f}")
print(f"   Average Recall: {overall_test_recall:.4f}")

# Check for overfitting on test set
print(f"\n🔍 GENERALIZATION ANALYSIS:")
for i, target in enumerate(target_cols):
    val_f1 = final_df.loc[i, "Val_F1"]
    test_f1 = final_df.loc[i, "Test_F1"]
    gap = val_f1 - test_f1

    print(f"\n🎯 {target}:")
    print(f"   Validation F1: {val_f1:.4f}")
    print(f"   Test F1: {test_f1:.4f}")
    print(f"   Gap: {gap:+.4f}")

    if abs(gap) < 0.05:
        print(f"   ✅ Excellent generalization")
    elif abs(gap) < 0.1:
        print(f"   ✅ Good generalization")
    else:
        print(f"   ⚠️  Potential generalization issues")

In [ ]:
# Final visualizations for test set
print("📊 Generating Final Test Set Visualizations...")

# Test set confusion matrices
plot_confusion_matrices(y_test, y_test_pred_final, target_cols, "Final Test - ")

# Test set precision-recall curves
plot_precision_recall_curves(y_test, y_test_proba_final, target_cols, "Final Test - ")


# Performance comparison visualization
def plot_performance_comparison(final_df):
    """Plot performance comparison across train/val/test sets"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # F1 Scores
    x = np.arange(len(target_cols))
    width = 0.25

    axes[0].bar(x - width, final_df["Train_F1"], width, label="Train", alpha=0.8)
    axes[0].bar(x, final_df["Val_F1"], width, label="Validation", alpha=0.8)
    axes[0].bar(x + width, final_df["Test_F1"], width, label="Test", alpha=0.8)
    axes[0].set_xlabel("Target Classes")
    axes[0].set_ylabel("F1 Score")
    axes[0].set_title("F1 Score Comparison")
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(target_cols, rotation=45)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Precision
    axes[1].bar(x - width, final_df["Train_Precision"], width, label="Train", alpha=0.8)
    axes[1].bar(x, final_df["Val_Precision"], width, label="Validation", alpha=0.8)
    axes[1].bar(x + width, final_df["Test_Precision"], width, label="Test", alpha=0.8)
    axes[1].set_xlabel("Target Classes")
    axes[1].set_ylabel("Precision")
    axes[1].set_title("Precision Comparison")
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(target_cols, rotation=45)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # Recall
    axes[2].bar(x - width, final_df["Train_Recall"], width, label="Train", alpha=0.8)
    axes[2].bar(x, final_df["Val_Recall"], width, label="Validation", alpha=0.8)
    axes[2].bar(x + width, final_df["Test_Recall"], width, label="Test", alpha=0.8)
    axes[2].set_xlabel("Target Classes")
    axes[2].set_ylabel("Recall")
    axes[2].set_title("Recall Comparison")
    axes[2].set_xticks(x)
    axes[2].set_xticklabels(target_cols, rotation=45)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


# Plot performance comparison
plot_performance_comparison(final_df)

# Final summary and recommendations
print(f"\n🎉 FINAL MODEL SUMMARY & RECOMMENDATIONS")
print("=" * 80)

print(f"\n🏆 MODEL ACHIEVEMENTS:")
print(f"   ✅ Multi-label classification pipeline implemented")
print(f"   ✅ Text embeddings from SentenceTransformers integrated")
print(f"   ✅ Metadata features properly encoded and scaled")
print(f"   ✅ Hyperparameter optimization completed")
print(f"   ✅ Comprehensive evaluation across train/val/test sets")

print(f"\n📊 FINAL PERFORMANCE METRICS:")
for i, target in enumerate(target_cols):
    test_f1 = final_df.loc[i, "Test_F1"]
    test_precision = final_df.loc[i, "Test_Precision"]
    test_recall = final_df.loc[i, "Test_Recall"]

    print(f"   🎯 {target}:")
    print(
        f"      F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}"
    )

print(f"\n🔧 BEST HYPERPARAMETERS FOUND:")
for target, params in best_params.items():
    print(f"\n   🎯 {target}:")
    for param, value in params.items():
        print(f"      {param}: {value}")

print(f"\n💡 DEPLOYMENT RECOMMENDATIONS:")
print(f"   • Model shows good generalization to test set")
print(f"   • Consider ensemble methods for further improvement")
print(f"   • Monitor performance on new data and retrain periodically")
print(f"   • Implement threshold optimization for production use")
print(f"   • Set up A/B testing to validate real-world performance")

print(f"\n🚀 MODEL READY FOR PRODUCTION DEPLOYMENT!")
print("=" * 80)